# Assignment 2
## ISPR 23/24, third assignment
### by Andrea Marino (matr. 561935)

### Preliminary stuff

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sys
#from PIL import Image              # Python Image Library

from typing import List, Tuple

In [2]:
if(colab := 'google.colab' in sys.modules):
    from google.colab import drive
    drive.mount("/content/drive")
    BASE_PATH = "drive/MyDrive/ISPR/Compitini ISPR/Terzo Compitino/"
    sys.path.insert(0,BASE_PATH)
    #os.chdir(BASE_PATH)
    #!pip install -U torch           # update PyTorch
    #!pip install -U torchvision     # update torchvision
else:
    BASE_PATH = ""


In [3]:
import torch
from torch import nn

Let's also set PyTorch's seed to a fixed value, for reproducibility but also to get the same splits in the datasets we're going to create

In [4]:
torch.manual_seed(42)

Finally, we define the device.

In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Data, Dataset and Dataloaders

Since I already used the dataset for the first assignment, I don't need to download it again. I just have to fetch it from the right directory (in a way that deals with both Colab and my local install)

In [6]:
HORSE_PATH = os.path.join(BASE_PATH,"../Primo Compitino/weizmann_horse_db/horse/")
MASK_PATH = os.path.join(BASE_PATH,"../Primo Compitino/weizmann_horse_db/mask/")

HORSE_PATH, MASK_PATH

('../Primo Compitino/weizmann_horse_db/horse/',
 '../Primo Compitino/weizmann_horse_db/mask/')

We now need to create the class for the dataset, as required by PyTorch.

We can also apply transformations on our data. The principal transformation worth talking about has to deal with the batch size and the shape of the images. 

If in our `DataLoader`s the batch size is set to be greater than 1, then the images have to be of the same shape (height * width), otherwise the associated tensors can't be stacked in a single batch tensor. This is unfortunate, we can deal with it in three ways:
1. Always online learning algorithm (aka batch_size=1)
2. Resize the image
3. Add a padding of zeros to the smaller images, to bring them all to the same size as the bigger image in the dataset (and then let the model deal with the extra padding).

All three options are interesting for their own reasons, so writing a more general code that allows one to apply each of the ideas is worthwhile.

The first option allows to assess the model's performance on images of different sizes, an interesting challenge for sure. The second option is the most promising and most powerful one, as torchvision's tools seems to do a good job at resizing both the images and the mask, upon preliminary checking. The third option is pretty rough, but it might be interesting to see how the model performs in this case.

In [7]:
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import Lambda

class HorseDataset(Dataset):
    def __init__(self, 
                 image_path=HORSE_PATH,
                 mask_path=MASK_PATH,
                 transform=Lambda(lambda img: img/255), # normalize pixel values
                 target_transform=None,):
        self.img_dir = image_path
        self.mask_dir = mask_path
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        # We know that the answer is 327 but let's make it
        # more general and structured
        return len(os.listdir(self.img_dir))

    def __getitem__(self, idx):
        img_path, mask_path = self._horsePath(idx+1)
        img = read_image(img_path)      
        mask = read_image(mask_path)  
        img = img.float()
        mask = mask.float() # convert to float 
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            mask = self.target_transform(mask)
        mask[mask>0] = 1.0 # set all non-zero values to 1
        return img, mask
    
    def __iter__(self):
        for i in range(len(self)):
            yield self.__getitem__(i)    
    
    def _horsePath(self, h:int):
        '''
        Returns the path to the horse image
        whose number (in the filename) is h
        '''
        number = "0"*(2-int(np.log10(h)))+str(h)
        imgname = "horse" + number + ".png"
        img_path = self.img_dir + imgname
        mask_path = self.mask_dir + imgname
        return img_path, mask_path

To apply the transformations as described, we need the height and width of the biggest and smallest images in the dataset.

In [8]:
plain_dataset = HorseDataset()

MAX, MIN = (0,0), (np.inf,np.inf)
biggest, smallest = -1,-1

for i, (img, _) in enumerate(plain_dataset,start=1):
    img_h, img_w = tuple(img.shape[1:])
    if img_h*img_w > MAX[0]*MAX[1]:
        MAX = (img_h, img_w)
        biggest = i
    if img_h*img_w < MIN[0]*MIN[1]:
        MIN = (img_h, img_w)
        smallest = i

print(f"The biggest image is horse {biggest}, of size {MAX[0]}x{MAX[1]}")
print(f"The smallest image is horse {smallest}, of size {MIN[0]}x{MIN[1]}")

The biggest image is horse 65, of size 960x1280
The smallest image is horse 16, of size 148x231


In [9]:
image0, mask0 = plain_dataset[0]
print(f"Image shape: {image0.shape}, mask shape: {mask0.shape}")
print(f"Image type: {image0.dtype}, mask type: {mask0.dtype}")
print(f"Image values range: {torch.min(image0).item()} - {torch.max(image0).item()}")
print(f"Mask values range: {torch.min(mask0).item()} - {torch.max(mask0).item()}")

Image shape: torch.Size([3, 590, 800]), mask shape: torch.Size([1, 590, 800])
Image type: torch.float32, mask type: torch.float32
Image values range: 0.0 - 1.0
Mask values range: 0.0 - 1.0


#### Horse dataset of unchanged images

We create a dataset following the first option.

We can use the `random_split` to create the training set and the test set. 

20% of the whole dataset is held out as test set. I chose this approach over cross-validation to alleviate the computational burden.

In [10]:
from torch.utils.data import random_split

# Create the developement and test datasets
train_size = int(0.8 * len(plain_dataset))
test_size = len(plain_dataset) - train_size

training_plain_dataset, test_plain_dataset = random_split(plain_dataset, [train_size, test_size])

Now we can create the DataLoaders for the training and test data, using batch size of 1 since we applied no transformation

In [11]:
from torch.utils.data import DataLoader

# Attenzione:
BATCH_SIZE = 1

train_plain_dataloader = DataLoader(training_plain_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_plain_dataloader = DataLoader(test_plain_dataset, batch_size=BATCH_SIZE, shuffle=True)

#### Horse dataset of resized images

Let's see how the `HorseDataset` class allows us to create a dataset of resized images.

In [12]:
from torchvision.transforms import Resize

#resize_transform = lambda h,w: Lambda(lambda X: Resize((h,w))(X))
def resize_transform(height:int, width:int):
    return Lambda(lambda X: Resize((height,width))(X))

In [13]:
# We can resize to the smallest, or choose any other size
h, w = 256, 256

dataset_resized = HorseDataset(transform=resize_transform(h,w), 
                              target_transform=resize_transform(h,w))

DataLoader creation:


In [14]:
training_resized_dataset, test_resized_dataset = random_split(dataset_resized, [train_size, test_size])

BATCH_SIZE = 4

train_resized_dataloader = DataLoader(training_resized_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_resized_dataloader = DataLoader(test_resized_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [15]:
print(f"Training: n° images is {len(train_resized_dataloader.dataset)}, n° of minibatches is: {len(train_resized_dataloader)}")
print(f"Test: n° images is {len(test_resized_dataloader.dataset)}, n° of minibatches is: {len(test_resized_dataloader)}")

for X, y in train_resized_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape, X.dtype)
    print("Shape of y: [N, C, H, W]", y.shape, y.dtype)
    break

for i, (X, y) in enumerate(train_resized_dataloader):
    assert torch.min(y) >= 0 and torch.max(y) <= 1, f"Error at minibatch {i}: targets should be in [0, 1] for BCELoss. Found min {torch.min(y)} and max {torch.max(y)}"


Training: n° images is 261, n° of minibatches is: 66
Test: n° images is 66, n° of minibatches is: 17
Shape of X [N, C, H, W]:  torch.Size([4, 3, 256, 256]) torch.float32
Shape of y: [N, C, H, W] torch.Size([4, 1, 256, 256]) torch.float32


In [16]:
torch.min(y), torch.max(y)

(tensor(0.), tensor(1.))

#### Horse dataset of padded images

Lastly, we create a dataset of padded images using the `HorseDataset` class.

Instead of passing a fixed amount of padding to the transformation, we pass it the target height and width of the image.

In [17]:
from torchvision.transforms import Pad

#pad_transform = lambda target_h, target_w: Lambda(lambda X: Pad((X.shape[1]-target_h, X.shape[2]-target_w), fill=0)(X))
def pad_transform(target_h:int, target_w:int):
    return Lambda(lambda X: Pad((X.shape[1]-target_h, X.shape[2]-target_w), fill=0)(X))

In [18]:
h, w = MAX

dataset_padded = HorseDataset(transform=pad_transform(h,w), 
                              target_transform=pad_transform(h,w))

# Check that things are ok

DataLoader creation:

In [19]:
training_padded_dataset, test_padded_dataset = random_split(dataset_padded, [train_size, test_size])

BATCH_SIZE = 4

train_padded_dataloader = DataLoader(training_padded_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_padded_dataloader = DataLoader(test_padded_dataset, batch_size=BATCH_SIZE, shuffle=True)

### The model

The task is semantic segmentation of the horse from the background. The chosen model for this task is a *U-shaped network* that uses convolution and transposed convolutions. Given the context, such a network could be referred as a horseshoe network.

Our horseshoe network has two parts: an encoder and a decoder. For the sake of simplicity (and cleanness of code), these are split into two parts.

#### Encoder

Let's start with the `HorseshoeEncoder`. Even though that's somewhat ambiguous and deplorable (the ambiguity is also in the literature), inside a convolutional layer three things will happen:
- A given number of convolutional filters will be applied
- The (batch of) resulting feature maps will be normalized
- An activation function is applied

The `conv_layer` nested function returns a convolutional layer as a `Sequential` container. 

We can specify the structure of our decoder via the `architecture` parameter of the constructor. This parameter is a list of tuple, each tuple being relative to a *block*. 

In this context, a block is just a sequence of convolutional filters followed by a max pooling. More precisely, each tuple in the list specifies how many convolutional layers we want to stack before applying the pooling (1st component of the tuple), and how many convolutional filters there are in each layer inside the block (2nd component of the tuple).

So, in abstract terms, our encoder is just a sequence of blocks. In practice, each block is again a Sequential module, and the encoder's layers are implemented as a `nn.ModuleDict`. A dictionary of modules is used because we need to keep track of the pooling indices of the maximum element. 

In [20]:
class HorseshoeEncoder(nn.Module):
    def __init__(self,
                 architecture:List[Tuple]=[(2,16),(2,32),(3,64),(3,128),(3,128)],
                 in_channels:int=3,) -> None:
        super(HorseshoeEncoder, self).__init__()

        def conv_layer(in_channels, out_channels, *args, **kwargs):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, *args, **kwargs),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
        
        self.layers = nn.ModuleDict()

        in_channels = in_channels  # initial number of input channels
        for i, (n_blocks, out_channels) in enumerate(architecture):
            conv_layers = []
            for _ in range(n_blocks):
                conv_layers.append(conv_layer(in_channels, out_channels, kernel_size=3, stride=1, padding=0))
                in_channels = out_channels
            conv_block = nn.Sequential(*conv_layers)
            # We add a MaxPooling layer after the prescribed number of conv layers
            self.layers.add_module(f'conv_block_{i}',nn.ModuleList([conv_block, nn.MaxPool2d(2,2,return_indices=True)]))

        self.conv1x1 = nn.Conv2d(in_channels, 256, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        pooling_indices = []
        input_sizes = []
        #for block in self.layers.values():
        for i in range(len(self.layers)):
            conv_block, pooling_layer = self.layers[f'conv_block_{i}']
            # Apply the convolutional layer
            x = conv_block(x)
            # we need to keep track of the input size
            # before the pooling, to do the unpooling correctly at decoding time
            input_sizes.append(x.size())
            # Apply the pooling layer
            x, indices = pooling_layer(x)
            # We need to keep track of the indices for the unpooling
            pooling_indices.append(indices)
        x = self.conv1x1(x)

        return x, pooling_indices, input_sizes

#### Decoder

The decoder is the mirror image of the encoder. We pass to it the same architecture parameter that was passed to the encoder, and the network is built from right to left in a kind of unintuitive fashion that allows to keep the simmetry and clarity of the code.

In [21]:
class HorseshoeDecoder(nn.Module):
    def __init__(self,
                 architecture:List[Tuple]=[(2,16),(2,32),(3,64),(3,128),(3,128)],
                 out_channels:int=1,) -> None:
        super(HorseshoeDecoder, self).__init__()

        def transp_conv_layer(in_channels, out_channels, *args, **kwargs):
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, *args, **kwargs),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
        
        self.layers = nn.ModuleDict()

        out_channels = out_channels     # desired n° of output channels
        arch_len = len(architecture)
        for i, (n_blocks, in_channels) in enumerate(architecture):
            transp_conv_layers = []
            for _ in range(n_blocks):
                transp_conv_layers.insert(0,transp_conv_layer(in_channels, out_channels, kernel_size=3, stride=1, padding=0))
                out_channels = in_channels
            transp_conv_block = nn.Sequential(*transp_conv_layers)
            # Since we're building the architecture from right to left, we need to add 
            # (more precisely, index) the modules in reversed order in the dictionary
            self.layers.add_module(f'transp_conv_block_{arch_len-i-1}',nn.ModuleList([nn.MaxUnpool2d(2,2), transp_conv_block])) 

        self.Tconv1x1 = nn.ConvTranspose2d(256, in_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x, pooling_indices, input_sizes):
        x = self.Tconv1x1(x)
        #for block in reversed(self.layers.values()):
        for i in range(len(self.layers)):
            unpooling_layer, transp_conv_block = self.layers[f'transp_conv_block_{i}']
            # First the unpooling (which needs the indices used for pooling and the size of the input)...
            x = unpooling_layer(x, pooling_indices.pop(), output_size=input_sizes.pop())
            # ... then the transposed convolution!
            x = transp_conv_block(x)

        return x

#### The full `HorseshoeNetwork`

Since we've separately written the encoder and the decoder, to create our horseshoe network we just have to put them together, basically.

In [22]:
class HorseshoeNetwork(nn.Module):
    def __init__(self,
                 in_channels:int=3,
                 out_channels:int=1,
                 architecture:List[Tuple]=[(2,16),(2,32),(3,64),(3,128),(3,128)],):
        super(HorseshoeNetwork, self).__init__()

        # Convolutional layers
        self.encoder = HorseshoeEncoder(architecture=architecture,in_channels=in_channels)

        # Transpose conv. layers
        self.decoder = HorseshoeDecoder(architecture=architecture,out_channels=out_channels)
        
    def forward(self, x):
        # Encoding
        x, pooling_indices, input_sizes = self.encoder(x)

        # Decoding
        x = self.decoder(x, pooling_indices, input_sizes)

        # Final sigmoid (classification)
        x = torch.sigmoid(x)
        return x

In [23]:
# Create our HorseshoeNetwork model
model = HorseshoeNetwork()
model.to(device)

HorseshoeNetwork(
  (encoder): HorseshoeEncoder(
    (layers): ModuleDict(
      (conv_block_0): ModuleList(
        (0): Sequential(
          (0): Sequential(
            (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
            (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (1): Sequential(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
            (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (conv_block_1): ModuleList(
        (0): Sequential(
          (0): Sequential(
            (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (1): Sequen

Unfortunately the transposed convolutional blocks are listed in reversed order in the decoder. 

### Training(-test) cycle

Now that we have the model and the dataset, it's time to write the training cycle.

Define the loss function and the optimizer

In [24]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [25]:
def train(dataloader, model, loss_fn, optimizer) -> None:
    model.train()
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)

        ## Debugging: Check the data range
        assert torch.min(y) >= 0 and torch.max(y) <= 1, "Targets should be in [0, 1] for BCELoss"

        ## Forward pass:
        # Compute prediction error
        pred = model(X)

        ## Debugging: Check the prediction range
        assert torch.min(pred) >= 0 and torch.max(pred) <= 1, "Predictions should be in [0, 1] after sigmoid"

        # Compute loss
        loss = loss_fn(pred, y)

        ## Backpropagation:
        # Zero out the gradient
        optimizer.zero_grad()
        # Compute the gradient (backward step)
        loss.backward()
        # apply weight update
        optimizer.step()
    

In [26]:
# To review, obviously
def evaluate(dataloader, model, loss_fn) -> float:
    num_batches = len(dataloader)
    test_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # compute accuracy
            
    test_loss /= num_batches
    #print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss

We can now build the training loop

In [27]:
# Change here to change the preprocessing strategy
train_dataloader = train_resized_dataloader
test_dataloader = test_resized_dataloader

In [28]:
EPOCHS = 5
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    train_loss = evaluate(train_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


/home/andrea/anaconda3/envs/ISPR-env/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/andrea/anaconda3/envs/ISPR-env/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 2
-------------------------------
Epoch 3
-------------------------------
Epoch 4
-------------------------------
Epoch 5
-------------------------------
Done!


#### Uffa

In [29]:
dataloader_cp = DataLoader(training_plain_dataset, batch_size=1, shuffle=True)

for i, (X, y) in enumerate(dataloader_cp):
    X, y = X.to(device), y.to(device)
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    print(f"X dtype: {X.dtype}, y dtype: {y.dtype}")
    pred = model(X)
    print(f"Prediction shape: {pred.shape}")
    print(f"Prediction dtype: {pred.dtype}")
    if i == 0:
        break

X shape: torch.Size([1, 3, 595, 504]), y shape: torch.Size([1, 1, 595, 504])
X dtype: torch.float32, y dtype: torch.float32
Prediction shape: torch.Size([1, 1, 595, 504])
Prediction dtype: torch.float32
